# Fine-tuning a Model

### Training

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation = True)

tokenized_dataset = raw_datasets.map(tokenize_function)

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors = 'tf')

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'token_type_ids'],
    label_cols = ['labels'],
    shuffle = True,
    batch_size = 8,
    collate_fn = data_collator
)

tf_validation_dataset = tokenized_dataset['validation'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'token_type_ids'],
    label_cols = ['labels'],
    shuffle = False,
    batch_size = 8,
    collate_fn = data_collator
)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [8]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer = 'adam',
    loss = SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

In [14]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

459/459 [==============================] - 133s 188ms/step - loss: 0.6584 - accuracy: 0.6611 - val_loss: 0.6252 - val_accuracy: 0.6838


### Improving Training Performance

- Transformer models benefit from a much lower learning rate than the default for Adam, which is 1e-3,. 5e-5 , which is some twenty times lower, is a much better starting point.

- We can slowly reduce the learning rate over the course of training.

In [16]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
batch_size =8
num_epochs =3

"""
The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied by the total number of epochs.
Note that the tf_train_dataset here is a batched tf.data.Dataset, not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
"""
num_train_steps = len(tf_train_dataset) * num_epochs

"""
Though it is a polynomical decay and can be used as a polynomial function,
Here we are trying it out as a linear function :p
"""
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate = lr_scheduler)

In [18]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(
    tf_train_dataset,
    validation_data = tf_validation_dataset,
    epochs = num_epochs
)

Epoch 1/3
459/459 [==============================] - 126s 186ms/step - loss: 0.5325 - accuracy: 0.7304 - val_loss: 0.3662 - val_accuracy: 0.8480
Epoch 2/3
459/459 [==============================] - 76s 165ms/step - loss: 0.2672 - accuracy: 0.8882 - val_loss: 0.3869 - val_accuracy: 0.8627
Epoch 3/3
459/459 [==============================] - 78s 169ms/step - loss: 0.0993 - accuracy: 0.9676 - val_loss: 0.4196 - val_accuracy: 0.8775


### Model Prediction

In [21]:
preds = model.predict(tf_validation_dataset)["logits"]

51/51 [==============================] - 6s 60ms/step


In [22]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(408, 2) (408,)


In [23]:
preds[0]

array([-3.140814 ,  3.2289577], dtype=float32)

In [25]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [26]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

{'accuracy': 0.8774509803921569, 'f1': 0.9137931034482758}